# Importing the required modules

In [2]:
import pandas as pd
import spacy
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")
# if it is not there then use python -m spacy download en_core_web_sm

# Load your dataset

In [5]:
df = pd.read_csv("../Data/intent_dataset_cleaned.csv", quotechar='"')
df = df.dropna()

In [6]:
# Display few entries
df.head()

,text,intent,response
0,text,intent,response
1,What does Tekworks do?,about,Tekworks is a global executive search firm spe...
2,Tell me about Tekworks.,about,Tekworks partners with businesses to provide e...
3,What services do you offer?,services,Tekworks offers Executive Hiring Leadership Hi...
4,Can you explain your executive hiring services?,services,Their Executive Hiring service helps businesse...


# Preprocessing 

In [7]:
def preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

# Apply preprocessing
df["clean_text"] = df["text"].apply(preprocess)

# Vectorization and model training


In [11]:
# Cell 4: Vectorization + Model Training
X = df["clean_text"]
y = df["intent"]

vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)

model = LogisticRegression(max_iter=1000)
model.fit(X_vec, y)

# Save model and vectorizer
joblib.dump(model, "../Models/tekbot_model.pkl")
joblib.dump(vectorizer, "../Models/tekbot_vectorizer.pkl")


['../Models/tekbot_vectorizer.pkl']

# Evaluation

In [12]:
# Cell 5: Evaluation (Optional)
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

                     precision    recall  f1-score   support

              about       1.00      0.50      0.67         4
           approach       0.00      0.00      0.00         3
            clients       0.00      0.00      0.00         2
            contact       1.00      1.00      1.00        12
           founders       0.00      0.00      0.00         1
             growth       0.88      0.88      0.88         8
     hiring_options       1.00      0.33      0.50         6
         industries       0.00      0.00      0.00         1
             intent       0.00      0.00      0.00         1
               jobs       0.00      0.00      0.00         4
        partnership       0.00      0.00      0.00         1
recruitment_process       0.94      1.00      0.97        16
           services       0.47      0.96      0.63        23
    talent_sourcing       1.00      0.60      0.75         5
           why_good       0.83      0.71      0.77         7
             why_us    

/Users/naresh/Desktop/Projects/TekWorks-chatbot/Chat-bot/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/naresh/Desktop/Projects/TekWorks-chatbot/Chat-bot/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/naresh/Desktop/Projects/TekWorks-chatbot/Chat-bot/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

# Save the model and vectorizer

In [13]:
# Cell 6: Intent + Response Prediction Function
# Build dictionary to map intent → response
intent_responses = df.groupby("intent")["response"].first().to_dict()

def predict_intent_and_respond(user_input):
    cleaned = preprocess(user_input)
    vect_input = vectorizer.transform([cleaned])
    predicted_intent = model.predict(vect_input)[0]
    response = intent_responses.get(predicted_intent, "Sorry, I don't have an answer for that.")
    return predicted_intent, response

# Interactive Test
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    intent, response = predict_intent_and_respond(query)
    print(f"🤖 TekBot (Intent: {intent}): {response}")


🤖 TekBot (Intent: about): Tekworks is a global executive search firm specializing in leadership and senior technical hires across industries. They help organizations build high-impact teams through tailored recruitment services.
🤖 TekBot (Intent: jobs): You can apply for a job at Tekworks by visiting the Careers page at www.tekworks.in/careers and filling out the contact form with your details and resume.
🤖 TekBot (Intent: clients): Tekworks works with a diverse set of clients across industries like healthcare finance technology e-commerce telecom and retail.
